In [1]:
import numpy as np
import sys
from urllib.request import urlopen
import urllib
import pandas as pd
Cleveland_data_URL = 'http://archive.ics.uci.edu/ml/machine-learning-databases/heart-disease/processed.hungarian.data'
np.set_printoptions(threshold=sys.maxsize) 

In [2]:
names = ['age', 'sex', 'cp', 'trestbps', 'chol', 'fbs', 'restecg', 'thalach', 'exang','oldpeak','slope','ca','thal',
'heartdisease']
heartDisease = pd.read_csv(urlopen(Cleveland_data_URL), names = names)
heartDisease.head()
del heartDisease['ca']
del heartDisease['slope']
del heartDisease['thal']
del heartDisease['oldpeak']

In [3]:
heartDisease = heartDisease.replace('?', np.nan)
heartDisease.dtypes
heartDisease.columns

Index(['age', 'sex', 'cp', 'trestbps', 'chol', 'fbs', 'restecg', 'thalach',
       'exang', 'heartdisease'],
      dtype='object')

In [4]:
from pgmpy.models import BayesianModel
from pgmpy.estimators import MaximumLikelihoodEstimator, BayesianEstimator

ModuleNotFoundError: No module named 'pgmpy'

In [ ]:
model = BayesianModel([('age','trestbps'), ('age','fbs'), ('sex', 'trestbps'), ('sex', 'trestbps'),('exang','trestbps'),('trestbps','heartdisease'),('fbs','heartdisease'),('heartdisease','restecg'),('heartdisease','thalach'),('heartdisease','chol')])                                                                                                        

In [31]:
model.fit(heartDisease, estimator=MaximumLikelihoodEstimator)
print(model.get_cpds('age'))
print(model.get_cpds('chol'))
print(model.get_cpds('sex'))
model.get_independencies()

+---------+------------+
| age(28) | 0.00383142 |
+---------+------------+
| age(29) | 0.00383142 |
+---------+------------+
| age(30) | 0.00383142 |
+---------+------------+
| age(31) | 0.00766284 |
+---------+------------+
| age(32) | 0.0153257  |
+---------+------------+
| age(33) | 0.00766284 |
+---------+------------+
| age(34) | 0.0153257  |
+---------+------------+
| age(35) | 0.0191571  |
+---------+------------+
| age(36) | 0.0191571  |
+---------+------------+
| age(37) | 0.0306513  |
+---------+------------+
| age(38) | 0.0191571  |
+---------+------------+
| age(39) | 0.0344828  |
+---------+------------+
| age(40) | 0.0191571  |
+---------+------------+
| age(41) | 0.0383142  |
+---------+------------+
| age(42) | 0.0268199  |
+---------+------------+
| age(43) | 0.0421456  |
+---------+------------+
| age(44) | 0.0268199  |
+---------+------------+
| age(45) | 0.0229885  |
+---------+------------+
| age(46) | 0.045977   |
+---------+------------+
| age(47) | 0.0344828  |


(age _|_ sex, exang)
(age _|_ exang | sex)
(age _|_ restecg, thalach, chol | heartdisease)
(age _|_ sex | exang)
(age _|_ sex, exang | fbs)
(age _|_ restecg, thalach, chol | sex, heartdisease)
(age _|_ exang | sex, fbs)
(age _|_ restecg, thalach, chol | trestbps, heartdisease)
(age _|_ restecg, thalach, chol | exang, heartdisease)
(age _|_ thalach, chol | restecg, heartdisease)
(age _|_ restecg, chol | heartdisease, thalach)
(age _|_ restecg, thalach, chol | heartdisease, fbs)
(age _|_ restecg, thalach | heartdisease, chol)
(age _|_ restecg, heartdisease, thalach, chol | trestbps, fbs)
(age _|_ sex | exang, fbs)
(age _|_ restecg, thalach, chol | sex, heartdisease, trestbps)
(age _|_ restecg, thalach, chol | sex, exang, heartdisease)
(age _|_ thalach, chol | sex, restecg, heartdisease)
(age _|_ restecg, chol | sex, heartdisease, thalach)
(age _|_ restecg, thalach, chol | sex, heartdisease, fbs)
(age _|_ restecg, thalach | sex, heartdisease, chol)
(age _|_ restecg, heartdisease, thalach,

In [32]:
from pgmpy.inference import VariableElimination
HeartDisease_infer = VariableElimination(model)

In [33]:
q = HeartDisease_infer.query(variables=['heartdisease'], evidence={'age': 28})
print(q)
q = HeartDisease_infer.query(variables=['heartdisease'], evidence={'chol': 100})
print(q)

Eliminating: chol: 100%|██████████| 7/7 [00:00<00:00, 117.26it/s]


+-----------------+---------------------+
| heartdisease    |   phi(heartdisease) |
+=================+=====================+
| heartdisease(0) |              0.6390 |
+-----------------+---------------------+
| heartdisease(1) |              0.3610 |
+-----------------+---------------------+


C:\Users\hp\anaconda3\lib\site-packages\pgmpy\factors\discrete\DiscreteFactor.py:518: UserWarning: Found unknown state name. Trying to switch to using all state names as state numbers
  warn(
Eliminating: fbs: 100%|██████████| 7/7 [00:00<00:00, 24.48it/s]     

+-----------------+---------------------+
| heartdisease    |   phi(heartdisease) |
+=================+=====================+
| heartdisease(0) |              1.0000 |
+-----------------+---------------------+
| heartdisease(1) |              0.0000 |
+-----------------+---------------------+
